In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
import os
import json
import torch
import yaml
import numpy as np
import cv2
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
from nerfstudio.engine.trainer import Trainer
from nerfstudio.utils.eval_utils import eval_setup

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
os.chdir('/workspace')
MODEL_PATH = "/workspace/outputs/5_spins_with_noise/plane-nerf/2023-11-27_101812"

In [4]:
config_path = os.path.join(MODEL_PATH, "config.yml")

config = yaml.load(Path(config_path).read_text(), Loader=yaml.Loader)

config, pipeline, checkpoint_path, _ = eval_setup(
                        Path(config_path),
                        test_mode="inference",
                    )

outputs/5_spins_with_noise/plane-nerf/2023-11-27_101812/nerfstudio_models


Loading latest checkpoint from load_dir

✅ Done loading checkpoint from 
outputs/5_spins_with_noise/plane-nerf/2023-11-27_101812/nerfstudio_models/step-000009999.ckpt

In [11]:
#Camera object: generate_ray command to create
#RayBundle object

pipeline.eval()
pipeline.datamanager.setup_eval()

psnr = []
ssim = []
lpips = []

for camera, batch in pipeline.datamanager.fixed_indices_eval_dataloader:
    outputs = pipeline.model.get_outputs_for_camera(camera=camera)
    height, width = camera.height, camera.width
    num_rays = height * width
    metrics_dict, _ = pipeline._model.get_image_metrics_and_images(outputs, batch)
    
    psnr.append(metrics_dict["psnr"])
    ssim.append(metrics_dict["ssim"])
    lpips.append(metrics_dict["lpips"])
    
print("PSNR: ", np.mean(psnr))
print("SSIM: ", np.mean(ssim))
print("LPIPS: ", np.mean(lpips))

Setting up evaluation dataset...

Caching all 25 images.

Output()

/home/user/.local/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:70: FutureWarning: Importing `spectral_angle_mapper` from `torchmetrics.functional` was deprecated and will be removed in 2.0. Import `spectral_angle_mapper` from `torchmetrics.image` instead.
  _future_warning(


PSNR:  12.474211120605469
SSIM:  0.020662419199943542
LPIPS:  1.0490009546279908
